<h3> The following is a copy  of the dashboard code created through coursera Skills Network Labs  </h3>

<p>Note:  Since the Lab cannot be downloaded and uploaded into Watson, this notebook serves only to show the code and process of the lab</p>

In [5]:
pip install pandas dash


     |████████████████████████████████| 7.4 MB 17.7 MB/s eta 0:00:01
     |████████████████████████████████| 27.7 MB 70.8 MB/s eta 0:00:01
     |████████████████████████████████| 95 kB 9.2 MB/s  eta 0:00:01
     |████████████████████████████████| 357 kB 37.0 MB/s eta 0:00:01
  Created wheel for dash-core-components: filename=dash_core_components-2.0.0-py3-none-any.whl size=3804 sha256=0a9deb2bf2702ba439ffb287b87196eda335e5b2de562713f86490966c300be9
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/52/e4/f3/16724791571a955a46d54650510c98c04ab7d339626aee27cc
  Created wheel for dash-html-components: filename=dash_html_components-2.0.0-py3-none-any.whl size=4069 sha256=fd89d841c4c8ee98b9dc3d021eeed5794f0097342d3740c54e0817ab1c189839
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/73/d8/8d/92f612c03c895f19bcc56a6c54be7bb41aaa698012a5624f60
  Created wheel for dash-table: filename=dash_table-5.0.0-py3-none-any.whl size=3897 sha256=239673bdf8bbd35345283d21ab5e4ae3d29803c090c03ca9772a

In [6]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

/tmp/wsuser/ipykernel_1059/1399112611.py:4: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
/tmp/wsuser/ipykernel_1059/1399112611.py:5: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


In [7]:
path = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv'
df = pd.read_csv(path)
df.to_csv('spacex_launch_dash.csv')

In [8]:
#The following code is what creates the Dashboard shown in the powerpoint presentation.
#To replicate, copy the framework below and use the file in the previous cells of this notebook.

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                html.Div
                                    ([
                                        dcc.Dropdown
                                            (
                                                id = 'site-dropdown',
                                                options = 
                                                    [
                                                        {'label': 'ALL Sites', 'value':'ALL'},
                                                        {'label': 'CCAFS LC-40', 'value':'CCAFS LC-40'},
                                                        {'label': 'KSC LC-39A', 'value':'KSC LC-39A'},
                                                        {'label': 'VAFB SLC-4E', 'value':'VAFB SLC-4E'},
                                                        {'label': 'CCAFS SLC-40', 'value':'CCAFS SLC-40'},
                                                    ],
                                                value='ALL',
                                                placeholder='Select Site',
                                                searchable=True
                                            )
                                    ]),    
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div([dcc.Graph(id='success-pie-chart')]),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
  
                                dcc.RangeSlider
                                    (
                                        id='payload-slider',
                                        min=0, 
                                        max=10000, 
                                        step=1000,
                                        marks=
                                            {
                                                0: '0',
                                                1000:'1000',
                                                2000:'2000',
                                                3000:'3000',
                                                4000:'4000',
                                                5000:'5000',
                                                6000:'6000',
                                                7000:'7000',
                                                8000:'8000',
                                                9000:'9000',
                                                10000:'10000',
                                            },
                                        value=[0, 1000]
                                    ),
  
                                html.Br(),
                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div([dcc.Graph(id='success-payload-scatter-chart')])

                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output('success-pie-chart', 'figure'),
              Input('site-dropdown', 'value'))

def get_pie_chart(value):
    
    if value == 'ALL':
        A= spacex_df[spacex_df['class']==1]
        figure = px.pie(
                data_frame=A, 
                values='class', 
                names='Launch Site',
                title='All Sites'
            )
        
    else:
        filtered_df = spacex_df[['Launch Site', 'class']]
        B = filtered_df.groupby('Launch Site')['class'].value_counts().reset_index(name='counts')
        figure = px.pie(
                data_frame=B[B['Launch Site'] == value],
                values='counts', 
                names='class',
                color='class',
                color_discrete_map={1: 'rgb(204,255,204)', 0: 'rgb(229,204,255)'},
                title=value
            )   
    return figure
        # return the outcomes piechart for a selected site
        
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output('success-payload-scatter-chart', 'figure'),
              Input('site-dropdown', 'value'),
              Input('payload-slider', 'value')
             )  
def get_scatter_chart(site,payload):
    df = spacex_df[['Launch Site', 'Payload Mass (kg)', 'Booster Version Category' , 'class']]
    A = df.groupby(['Launch Site', 'Payload Mass (kg)', 'Booster Version Category'])['class'].value_counts().reset_index(name='counts')
    B =A[A['Payload Mass (kg)'].between(payload[0],payload[1])]
    if site =='ALL':
        
        fig = px.scatter(
                data_frame =B,
                x = 'Payload Mass (kg)',
                y = 'class',
                color = 'Booster Version Category'         
            )
    else:
        A = df[df['Launch Site']==site]
        B=A[A['Payload Mass (kg)'].between(payload[0],payload[1])]
        C = B.groupby(['Launch Site', 'Payload Mass (kg)', 'Booster Version Category'])['class'].value_counts().reset_index(name='counts')

        fig = px.scatter(
                data_frame =C,
                x = 'Payload Mass (kg)',
                y = 'class',
                color = 'Booster Version Category'         
            )

    return fig

# Run the app
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
